In [23]:
import numpy as np
from emukit.core.initial_designs.latin_design import LatinDesign
from emukit.core import ParameterSpace, ContinuousParameter
from simulator import MainSimulator
from tqdm import tqdm
import statistics

main_simulator = MainSimulator()

In [24]:
def target_function_list(X, repetition=1):
    Y = []
    for x in X:
        mutation_rates = {
            "size": x[0],
            "speed": x[1],
            "vision": x[2],
            "aggression": x[3]
        }
        items = []
        #total = 0
        for i in range(repetition):
            days_survived, log = main_simulator.run(mutation_rates, debug_info=DebugInfo(
                period=10, should_display_day=False, should_display_grid=False, should_display_traits=False), max_days=10000)
            items.append(days_survived)
            #total += days_survived
        Y.append(statistics.median(items))
        #Y.append(total//repetition)
    return np.array(Y)[:,np.newaxis]

In [26]:
space = ParameterSpace([ContinuousParameter('size', 0, 2),
                            ContinuousParameter('speed', 0, 2),
                            ContinuousParameter('vision', 0, 2),
                            ContinuousParameter('aggression', 0, 2)])

design = LatinDesign(space) # Collect random points
X = design.get_samples(10)

In [27]:
Y = []
for x in tqdm(X):
    y = target_function_list(np.array([x]),1)
    Y.append(y[0])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [05:02<00:00, 30.24s/it]


In [33]:
with open('dataset_test/mutation_rates.npy', 'wb') as f:
  np.save(f, np.array(X))
with open('dataset_test/simulated_years_of_survival.npy', 'wb') as f:
  np.save(f, np.array(Y))